In [3]:
# Import Libraries
import numpy as np
from pyspark.sql import SQLContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import pandasql as ps
import pyarrow
import boto3
import pyodbc
import ast
import io
import string
import re
from datetime import date,timedelta
from IPython.display import display
import json
import random
import time
import pandasql as ps
pd.options.mode.chained_assignment = None
from pyspark.sql.types import IntegerType,StructType,StructField,StringType
from pyspark.sql.functions import lit,array, create_map, struct
from pyspark.sql.functions import col, concat, regexp_extract, when,max as max_,min as min_
from pyspark.sql.functions import date_format
from pyspark.sql.functions import udf,col
from datetime import datetime,timedelta
import traceback
from pymongo import MongoClient
import csv
sc.addPyFile("s3n://c1-data-lake/ETL/ETL_Dependency/etlconfig.py")
sc.addPyFile("s3://c1-data-lake/ETL/UDF/prod/Udf.py")
sc.addPyFile("s3n://c1-data-lake/ETL/psqlconfig/psqlconfig.py")
import Udf
import etlconfig
import psqlconfig
import pyarrow


In [4]:
def getCurrentDate(delta):
    todaydate = date.today()-timedelta(days=delta)
    return todaydate
getCurrentDate(5)


In [ ]:
def getDeltaDate(delta,fileDate):
    daltadate=datetime.strptime(fileDate, '%m-%d-%Y')
    return daltadate.date()-timedelta(days=delta)

    

In [14]:
def getdeltaDate(delta,file_date):
    ddate=datetime.strptime(file_date,'%m-%d-%Y')
    daldate=ddate.date()-timedelta(days=delta)
    return daldate
getdeltaDate(2,"02-21-2023")
     

In [40]:
def querytoDataframe(Query):
    #cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.100.11.15;DATABASE=datalake;UID=agaur;PWD=Pune@123$A')
    cnxn = pyodbc.connect(etlconfig.pyodbc_datalake_prod)
    querystring_details = str(Query)
    try:
        dataframe_details = pd.read_sql(querystring_details, cnxn)
    except:
        print(querystring_details)
    cnxn.close()
    return dataframe_details
#querytoDataframe('select * from master.dealer')


In [20]:
querytoDataframe('select * from master.dealer')

In [24]:
# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.100.11.15;DATABASE=datalake;UID=agaur;PWD=Pune@123$A')
cnxn = pyodbc.connect(etlconfig.pyodbc_datalake_prod)
querystring_details = str('select * from master.dealer')

dataframe_details = pd.read_sql(querystring_details, cnxn)

In [32]:

%%configure -f
{ "driverMemory": "5G", "conf" : {"spark.executor.memory":"5g","spark.driver.memory":"5g","spark.memory.offHeap.enabled":"True","spark.memory.offHeap.size":"5g","spark.driver.maxResultSize":"2g","spark.rpc.message.maxSize": 1500,"spark.akka.frameSize":64,"spark.speculation":"true",
                                   "spark.speculation.multiplier":2,"spark.speculation.quantile":0},
 "jars": ["s3://c1-data-lake/jars/postgresql-42.3.1.jar","s3://c1-data-lake/jars/mongo-java-driver-3.4.2.jar","s3://c1-data-lake/jars/mongodb-driver-core-3.4.2.jar","s3://c1-data-lake/jars/mongo-spark-connector_2.11-2.2.1.jar","s3://c1-data-lake/jars/bson-3.4.2.jar"]}

In [ ]:
def clearSparkCache(spark,sqlCtx):
    sqlCtx.clearCache()
    spark.catalog.clearCache()

In [42]:
def getMongoDataBySpark(spark,collection,pipeline):
    client_collection=spark.read.format("com.mongodb.spark.sql.DefaultSource") \
                .option("uri",etlconfig.mongo_uri+"&connectTimeoutMS=900000") \
                .option("batchSize","50000") \
                .option("collection",collection).option("pipeline",pipeline).load()
    
    return client_collection

In [ ]:
def getClientDataFromMongo(spark,batch_id,oem_code,mongo_query,delta_date,natural_key_list):    
    nat_key_df_str = '","'.join(natural_key_list)
    nat_key_df_str = '"' +nat_key_df_str+ '"'
    oem_collection_code=""
    if (oem_code.lower()) in ['lexus','gm','kia','bmw','mini','mas','rry','mtr','mtb','fc','fca','dd','dd-bmw']:
        oem_collection_code = oem_code.lower() + "_vehicle360"
    else: 
        oem_collection_code = "dd_vehicle360"
    
    dynamic_query = str(mongo_query).replace("{dealer_code_array}",nat_key_df_str).replace("{date}",str(delta_date))
#     print(dynamic_query)
    return getMongoDataBySpark(spark,oem_collection_code,dynamic_query)


In [46]:
def getClientDataFromMongo(spark,batch_id,oem_code,mongo_query,delta_date,natural_key_list):  
    dynamic_query = str(mongo_query).replace("{dealer_code_array}",nat_key_df_str).replace("{date}",str(delta_date))
    return dynamic_query

In [22]:
def sparkReadTextFile(spark,sc,src_path):    
    dest_path="s3n://"+etlconfig.s3_bucket_name+"/"+src_path    
    rdf=spark.read.text(dest_path)
    return rdf

In [34]:
def sparkreadtext(spark,textfile_path):
    textfile_path="C:/Users/anjali.3/Downloads/anjali.csv"
    rdf=spark.read.csv(textfile_path)
    return rdf

    

In [ ]:
from pyspark.sql import SparkSession
 
spark = SparkSession.builder.appName(
    'Read All CSV Files in Directory').getOrCreate()
 
file2 = spark.read.csv('/content/*.csv', sep=',', 
                    inferSchema=True, header=True)

In [ ]:
def createClientFileByPartition(client_df,dest_path,dynamic_query,file_type_id,delimiter,partition,spark,sc):
    try:
        client_df.createOrReplaceTempView("clientfile")
        print(dest_path)        
#       // add natrual key to dynamic query to partition by natural key

        if(file_type_id==1):
            dynamic_query=dynamic_query.replace("from"," ,client_id from ").replace("group by","group by client_id, ")\
            .replace("CUSTOMER_CHECKSUM from (","CUSTOMER_CHECKSUM,client_id from (")

        if( file_type_id==2 or file_type_id==3 or file_type_id==5):
            print("211")
            dynamic_query=dynamic_query.replace("from clientfile"," ,client_id from clientfile").replace("group by","group by client_id, ")\
            .replace("CUSTOMER_CHECKSUM from (","CUSTOMER_CHECKSUM,client_id from (")
            print("213")
        elif(file_type_id==21):
            partition="1"
            opcodekeywords = S3filepathtoDataframe("ETL/PROD/OpcodeFiles/opcodeid_to_desc_all.txt","\t","infer")
            opcodekeywordsfinal = opcodekeywords[['opcode_category_id','category_name']]
            opcodekeywordsfinal['opcode_category_id'] = pd.to_numeric(opcodekeywordsfinal['opcode_category_id'], downcast='integer')
            ref_opcode=spark.createDataFrame(opcodekeywordsfinal)
            ref_opcode.createOrReplaceTempView("opcodekeywordsfinal")
            
#       // execute the dynamic query to get final data
        finaldataframe = spark.sql(dynamic_query)
        #print(dynamic_query)
        #print(finaldataframe)
        print(finaldataframe.count())
#         finaldataframe.count()
#       // write result data into S3 by partition
        sparkCsvWriteByPartition(sc,finaldataframe,partition,delimiter,dest_path)
        finaldataframe.unpersist()
    except Exception as ex:
        print(ex)
        
